# Exercise 2 - building vulnerbaility functions



unsing euqation 12 from https://gmd.copernicus.org/articles/12/3085/2019/

fij = (v3ij) / (1+v3ij), vij = max(vij−vthresh,0) / (vhalf−vthresh)

vthresh = 25.7 m s−1

vhalf = 74.7 m s−1


### import python libraries

In [39]:
import math

import pandas as pd

In [40]:
df_intensity = pd.read_csv('model_data/intensity_bin_dict.csv')

df_intensity

,intensity_bin_index,max_windspeed
0,1,45
1,2,50
2,3,55
3,4,60
4,5,65
5,6,70
6,7,75
7,8,80
8,9,85
9,10,90


In [33]:
# function to calculate damage factor f_ij from max windspeed v_ij
def get_damage_factor(v_ij,v_thresh,v_half):
    v_ij = max(v_ij - v_thresh,0) / (v_half - v_thresh)
    f_ij = (v_ij**3) / (1 + v_ij**3)
    return f_ij

In [34]:
#vulnerability functions
vulnerability_functions = {'residential': {'vulnerability_id':1, 'v_thresh': 25.7, 'v_half': 74.7}}

vulnerability_functions

{'residential': {'vulnerability_id': 1, 'v_thresh': 25.7, 'v_half': 74.7}}

In [42]:
# generate vulnerability file

mps_to_knots = 1.94384

v_lst = []
d_lst = []

for v in vulnerability_functions:
    v_id = vulnerability_functions[v]['vulnerability_id']
    v_thresh = vulnerability_functions[v]['v_thresh'] * mps_to_knots
    v_half = vulnerability_functions[v]['v_half'] * mps_to_knots
    
    for index,row in df_intensity.iterrows():
        intensity = row['intensity_bin_index']
        damage_bin = intensity
        ws_from = (row['max_windspeed'] - 2.5)
        ws_to = (row['max_windspeed'] + 2.5)
        damage_from = get_damage_factor(ws_from,v_thresh,v_half)
        damage_to = get_damage_factor(ws_to,v_thresh,v_half)
        
        v_row = [v_id,intensity,damage_bin]
        v_lst.append(v_row)
        
        d_row = [damage_bin,damage_from,damage_to]
        d_lst.append(d_row)
        
df_vuln = pd.DataFrame(data=v_lst,columns=['vulnerability_id','intensity_bin_index','damage_bin_index'],dtype='int')
df_vuln['prob']=1
df_vuln.to_csv('model_data/vulnerability.csv',index=False)

df_vuln

,vulnerability_id,intensity_bin_index,damage_bin_index,prob
0,1,1,1,1
1,1,2,2,1
2,1,3,3,1
3,1,4,4,1
4,1,5,5,1
5,1,6,6,1
6,1,7,7,1
7,1,8,8,1
8,1,9,9,1
9,1,10,10,1


In [43]:
# generate damage file

df_damage = pd.DataFrame(data=d_lst,columns=['damage_bin_index','damage_from','damage_to'])
df_damage['interpolation']=df_damage['damage_from'] + (df_damage['damage_to']-df_damage['damage_from'])/2
df_damage.to_csv('model_data/damage_bin_dict.csv',index=False)

df_damage

,damage_bin_index,damage_from,damage_to,interpolation
0,1,0.000000,0.000000,0.000000
1,2,0.000000,0.000019,0.000010
2,3,0.000019,0.000496,0.000258
3,4,0.000496,0.002279,0.001388
4,5,0.002279,0.006210,0.004244
5,6,0.006210,0.013085,0.009647
6,7,0.013085,0.023610,0.018348
7,8,0.023610,0.038356,0.030983
8,9,0.038356,0.057705,0.048030
9,10,0.057705,0.081819,0.069762
